In [1]:
import os

In [2]:
def convert_to_yolo_format(image_width, image_height, bbox):
    x1, y1 = bbox[0]
    x2, y2 = bbox[1]
    x3, y3 = bbox[2]
    x4, y4 = bbox[3]
    
    x_center = (x1 + x2 + x3 + x4) / 4.0
    y_center = (y1 + y2 + y3 + y4) / 4.0

    bbox_width = max(x1, x2, x3, x4) - min(x1, x2, x3, x4)
    bbox_height = max(y1, y2, y3, y4) - min(y1, y2, y3, y4)

    # Normalize the values
    x_center /= image_width
    y_center /= image_height
    bbox_width /= image_width
    bbox_height /= image_height

    return x_center, y_center, bbox_width, bbox_height

In [3]:
train_data_path = 'data/train_data'
train_bbox_path = 'data/train_bbox'
yolo_labels_path = 'yolo_labels'
os.makedirs(yolo_labels_path, exist_ok=True)

In [4]:
for filename in os.listdir(train_bbox_path):
    if filename.endswith('.txt'):
        image_id = os.path.splitext(filename)[0]
        bbox_file_path = os.path.join(train_bbox_path, filename)
        
        image_path = os.path.join(train_data_path, f"{image_id}.jpg")
        if not os.path.exists(image_path):
            continue
        
        from PIL import Image
        with Image.open(image_path) as img:
            image_width, image_height = img.size
        
        with open(bbox_file_path, 'r') as bbox_file:
            lines = bbox_file.readlines()
            bbox = []
            for line in lines:
                x, y = map(float, line.strip().split())
                bbox.append((x, y))
        
        x_center, y_center, bbox_width, bbox_height = convert_to_yolo_format(image_width, image_height, bbox)
        
        yolo_label_path = os.path.join(yolo_labels_path, f"{image_id}.txt")
        with open(yolo_label_path, 'w') as yolo_file:
            yolo_file.write(f"0 {x_center} {y_center} {bbox_width} {bbox_height}\n")